In [1]:
import pyspark as spark
from pyspark import SparkContext
# initialize a new Spark Context to use for the execution of the script
sc = SparkContext(appName="MY-APP-NAME", master="local[*]")
# prevent useless logging messages
sc.setLogLevel("ERROR")

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import FMClassifier
import numpy as np
from pyspark.mllib.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
import matplotlib.pyplot as plt

spark = SparkSession.builder \
    .master("local") \
    .appName("appName") \
    .getOrCreate()

In [3]:
def performance_metrics(predicted_values):
    eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

    #Root Mean Square Error
    rmse = eval.evaluate(predicted_values)
    print("RMSE: %.3f" % rmse)

    # Mean Square Error
    mse = eval.evaluate(predicted_values, {eval.metricName: "mse"})
    print("MSE: %.3f" % mse)

    # Mean Absolute Error
    mae = eval.evaluate(predicted_values, {eval.metricName: "mae"})
    print("MAE: %.3f" % mae)

    # r2 - coefficient of determination
    r2 = eval.evaluate(predicted_values, {eval.metricName: "r2"})
    print("r2: %.3f" %r2)

In [4]:
rf = DecisionTreeRegressor(seed = 69)

paramGrid = ParamGridBuilder()\
.addGrid(rf.maxBins, list(np.arange(2, 50, 2)))\
.addGrid(rf.maxDepth, list(np.arange(2, 28, 2)))\
.build()

tvs = TrainValidationSplit(estimator=rf,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2"), 
                           trainRatio=0.8)

In [5]:
data = spark.read.options(header='true', inferschema='true', delimiter=',').csv("data/df.csv")
data.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 1.712044550044049| 4.787830588906654|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.452311677836793|14.032851614737082|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.9888939958281715|  7.08478856513968|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 2.781362935991756| 18.63630522318597|
|      1238| 50| 50| -4.046307639606464|34.68797

In [6]:
#take distinct values of R and C
R_dumm = data.select('R').distinct().rdd.map(lambda r: r[0]).collect()
C_dumm = data.select('C').distinct().rdd.map(lambda r: r[0]).collect()

In [7]:
#build dummified columns
import pyspark.sql.functions as F

R_expr = [F.when(F.col("R") == r, 1).otherwise(0).alias("R_" + str(r)) for r in R_dumm]
C_expr = [F.when(F.col("C") == c, 1).otherwise(0).alias("C_" + str(c)) for c in C_dumm]

In [8]:
data = data.select("breath_ids", "m_u_in", "q_u_in", "m_u_out", "q_u_out", "m_pressure", "q_pressure", *R_expr + C_expr).drop('R_50', 'C_50')
data.show()

+----------+-------------------+------------------+------------------+------------------+------------------+------------------+----+---+----+----+
|breath_ids|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|R_20|R_5|C_20|C_10|
+----------+-------------------+------------------+------------------+------------------+------------------+------------------+----+---+----+----+
|       148| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 1.712044550044049| 4.787830588906654|   0|  0|   0|   1|
|       463| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.452311677836793|14.032851614737082|   0|  0|   0|   1|
|       471|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.9888939958281715|  7.08478856513968|   1|  0|   1|   0|
|       496| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 2.781362935991756| 18.636305

# Regression #1 
## Regress m and q parameters for the pressure attribute indepentently

## Regression m_pressure

In [9]:
features = ['breath_ids', 'R_5', 'R_20', 'C_10', 'C_20', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out']
lr_data = data.select(col('m_pressure').alias("label"), *features)

In [10]:
train, test = lr_data.randomSplit([0.8, 0.2], seed=69)

In [11]:
train.show(5)

+------------------+----------+---+----+----+----+-------------------+------------------+------------------+------------------+
|             label|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|
+------------------+----------+---+----+----+----+-------------------+------------------+------------------+------------------+
|-7.866856074833947|     88590|  0|   0|   0|   1| -25.41676240979036| 65.80159905394707| 3.594318209183017|-4.709952037521332|
|-7.663688146587497|    116649|  0|   0|   0|   1|-21.639086214137617| 54.87565125780993| 3.594773597953297|-4.709796481642885|
|-7.554157606521539|     20210|  0|   0|   0|   1| -21.45036564244344|53.970380395461575|  3.59429508158818|-4.713761437371403|
|-7.507951297643809|    124292|  0|   0|   0|   1|-21.908552023117497| 55.37320489748355| 3.594386690014932|-4.714122611281731|
|-7.476816741717444|      9323|  0|   0|   0|   1| -21.41138677291981| 53.63256385162418|3.5943172325696

In [12]:
#prepare the data and the pipeline
for_prediction = features.copy()
for_prediction.remove('breath_ids')
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
stages = [vectorAssembler, standardScaler, tvs]
pipeline = Pipeline(stages=stages)

### Models performances for m_pressure

In [13]:
model = pipeline.fit(train)
prediction = model.transform(test)

In [14]:
performance_metrics(prediction)

RMSE: 1.158
MSE: 1.342
MAE: 0.787
r2: 0.811


In [15]:
print('Best Param (MaxBins): ',rf.getMaxBins())
print('Best Param (MaxDepth): ',rf.getMaxDepth())

Best Param (MaxBins):  32
Best Param (MaxDepth):  5


### Get predicted m to the original dataset

In [16]:
m_pressure = model.transform(lr_data)
m_press_predicted = m_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

## Regression for q_pressure

In [17]:
lr_data = data.select(col('q_pressure').alias("label"), *features)

In [18]:
train, test = lr_data.randomSplit([0.8, 0.2], seed=69)

In [19]:
#prepare the data and the pipeline
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
stages = [vectorAssembler, standardScaler, tvs]
pipeline = Pipeline(stages=stages)

### Test performances on q_pressure

In [20]:
model = pipeline.fit(train)
prediction = model.transform(test)

In [21]:
print(for_prediction)

['R_5', 'R_20', 'C_10', 'C_20', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out']


In [22]:
performance_metrics(prediction)

RMSE: 1.304
MSE: 1.699
MAE: 0.880
r2: 0.957


In [23]:
print('Best Param (MaxBins): ',rf.getMaxBins())
print('Best Param (MaxDepth): ',rf.getMaxDepth())

Best Param (MaxBins):  32
Best Param (MaxDepth):  5


### Get predicted q to the original dataset 

In [24]:
q_pressure = model.transform(lr_data)
q_press_predicted = q_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

# Output from the first regression technique

In [25]:
df_regressed_1 = data.select(features)
df_regressed_1 = df_regressed_1.join(m_press_predicted, m_press_predicted.b_id == df_regressed_1.breath_ids).select(*df_regressed_1.columns, col('prediction').alias('m_pressure'))
df_regressed_1 = df_regressed_1.join(q_press_predicted, q_press_predicted.b_id == df_regressed_1.breath_ids).select(*df_regressed_1.columns, col('prediction').alias('q_pressure'))
df_regressed_1.show(5)

+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|
+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|1.4522563494115552| 5.887222668718062|
|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.952493656751781|14.168895301860081|
|       471|  0|   1|   0|   1|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|  2.68850908668299| 7.813178418407266|
|       496|  0|   1|   0|   1| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 4.7121919

# Regression #2
### 1. Regress the m parameter for pressure
### 2. Include the predicted value of m in the training set when predicting the q

In [26]:
data.show(5)

+----------+-------------------+------------------+------------------+------------------+------------------+------------------+----+---+----+----+
|breath_ids|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|R_20|R_5|C_20|C_10|
+----------+-------------------+------------------+------------------+------------------+------------------+------------------+----+---+----+----+
|       148| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 1.712044550044049| 4.787830588906654|   0|  0|   0|   1|
|       463| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.452311677836793|14.032851614737082|   0|  0|   0|   1|
|       471|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.9888939958281715|  7.08478856513968|   1|  0|   1|   0|
|       496| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 2.781362935991756| 18.636305

In [27]:
# Append to the original dataset the m_pressure field calculated from the step before
df_m = data.select(features)
df_m = data.join(m_press_predicted, m_press_predicted.b_id == df_m.breath_ids).select(*df_m.columns, col('prediction').alias('m_pressure'))
df_m.show(5)

+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|
+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+
|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|1.4522563494115552|
|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.952493656751781|
|       471|  0|   1|   0|   1|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|  2.68850908668299|
|       496|  0|   1|   0|   1| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 4.712191946387388|
|      1238|  0|   0|   0|   0| -4.046307639606464|34.687972375171306|3.5981683352545537|-4.872868892058

In [28]:
m_features = ['breath_ids', 'R_5', 'R_20', 'C_10', 'C_20', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out', 'm_pressure']
q = data.select('q_pressure', col('breath_ids').alias('b_id'))
lr_data = df_m.join(q, q.b_id == df_m.breath_ids).select(*df_m.columns, col('q_pressure').alias("label"))
lr_data.show(5)

+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|             label|
+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|1.4522563494115552| 4.787830588906654|
|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.952493656751781|14.032851614737082|
|       471|  0|   1|   0|   1|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|  2.68850908668299|  7.08478856513968|
|       496|  0|   1|   0|   1| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 4.7121919

In [29]:
train, test = lr_data.randomSplit([0.8, 0.2], seed=69)

In [30]:
#prepare the data and the pipeline
for_prediction = m_features.copy()
for_prediction.remove('breath_ids')
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
stages = [vectorAssembler, standardScaler, tvs]
pipeline = Pipeline(stages=stages)

## Regression for q_pressure

In [31]:
model = pipeline.fit(train)
prediction = model.transform(test)

In [32]:
performance_metrics(prediction)

RMSE: 1.366
MSE: 1.866
MAE: 0.905
r2: 0.952


In [33]:
prediction.show(5)

+----------+---+----+----+----+-------------------+-----------------+------------------+------------------+-------------------+------------------+--------------------+--------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|           q_u_in|           m_u_out|           q_u_out|         m_pressure|             label|   unscaled_features|            features|        prediction|
+----------+---+----+----+----+-------------------+-----------------+------------------+------------------+-------------------+------------------+--------------------+--------------------+------------------+
|       133|  1|   0|   1|   0|  -19.6014207492924|46.98668700460304| 3.594566274579692| -4.71231980483322|   6.94818932980579|18.158814395307186|[1.0,0.0,1.0,0.0,...|[2.13104243257100...|17.098833121645907|
|       177|  1|   0|   0|   0| -23.32346938584597|71.05051858227964|3.5843312532246725|-4.381444835800677| 3.1342895580162065|11.163875805026365|[1.0,0.0,0.0,0.0,...|[

## Output from the second regression technique

In [34]:
q_pressure.show(5)

+------------------+----------+---+----+----+----+-------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+
|             label|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|   unscaled_features|            features|        prediction|
+------------------+----------+---+----+----+----+-------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+
| 4.787830588906654|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|[0.0,0.0,1.0,0.0,...|[0.0,0.0,2.067645...| 5.887222668718062|
|14.032851614737082|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274|[0.0,0.0,1.0,0.0,...|[0.0,0.0,2.067645...|14.168895301860081|
|  7.08478856513968|       471|  0|   1|   0|   1|-1.10

In [35]:
q_pressure = model.transform(lr_data)
q_press_predicted = q_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

In [36]:
df_regressed_2 = data.select(m_features)
df_regressed_2 = df_regressed_2.join(q_press_predicted, q_press_predicted.b_id == df_regressed_2.breath_ids).select(*df_regressed_2.columns, col('prediction').alias('q_pressure'))
df_regressed_2.show(5)

+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|
+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 1.712044550044049| 5.777340379352523|
|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.452311677836793|14.042833228981776|
|       471|  0|   1|   0|   1|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.9888939958281715| 8.650926639695456|
|       496|  0|   1|   0|   1| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 2.7813629

# Regression #3 
### 1. Regress the q parameter for pressure 
### 2. Include the predicted value of q in the trianning set when predicting the m

In [37]:
# Append to the original dataset the q_pressure field calculated from the step before
df_q = data.select(features)
df_q = df_q.join(q_press_predicted, q_press_predicted.b_id == df_m.breath_ids).select(*df_q.columns, col('prediction').alias('q_pressure'))
df_q.show(5)

+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        q_pressure|
+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+
|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 5.777340379352523|
|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274|14.042833228981776|
|       471|  0|   1|   0|   1|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605| 8.650926639695456|
|       496|  0|   1|   0|   1| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 19.95691332133398|
|      1238|  0|   0|   0|   0| -4.046307639606464|34.687972375171306|3.5981683352545537|-4.872868892058

In [38]:
q_features = ['breath_ids', 'R_5', 'R_20', 'C_10', 'C_20', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out', 'q_pressure']
m = data.select('m_pressure', col('breath_ids').alias('b_id'))
lr_data = df_q.join(m, m.b_id == df_q.breath_ids).select(*df_q.columns, col('m_pressure').alias('label'))
lr_data.show(5)

+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        q_pressure|             label|
+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 5.777340379352523| 1.712044550044049|
|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274|14.042833228981776| 5.452311677836793|
|       471|  0|   1|   0|   1|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605| 8.650926639695456|2.9888939958281715|
|       496|  0|   1|   0|   1| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 19.956913

In [39]:
train, test = lr_data.randomSplit([0.8, 0.2], seed=69)

In [40]:
#prepare the data and the pipeline
for_prediction = q_features.copy()
for_prediction.remove('breath_ids')
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
stages = [vectorAssembler, standardScaler, tvs]
pipeline = Pipeline(stages=stages)

## Regression for m_pressure

In [41]:
model = pipeline.fit(train)
prediction = model.transform(test)

In [42]:
print(for_prediction)

['R_5', 'R_20', 'C_10', 'C_20', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out', 'q_pressure']


In [43]:
performance_metrics(prediction)

RMSE: 1.230
MSE: 1.512
MAE: 0.833
r2: 0.777


In [44]:
prediction.show(5)

+----------+---+----+----+----+-------------------+-----------------+------------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|           q_u_in|           m_u_out|           q_u_out|        q_pressure|              label|   unscaled_features|            features|        prediction|
+----------+---+----+----+----+-------------------+-----------------+------------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+
|       133|  1|   0|   1|   0|  -19.6014207492924|46.98668700460304| 3.594566274579692| -4.71231980483322|17.098833121645907|  4.694679995581807|[1.0,0.0,1.0,0.0,...|[2.13104243257100...| 5.906446016617122|
|       177|  1|   0|   0|   0| -23.32346938584597|71.05051858227964|3.5843312532246725|-4.381444835800677|11.529143278975685| 3.0100866207852106|[1.0,0.0,0.0,0.0,...|[

## Output from the second regression technique

In [45]:
m_pressure = model.transform(lr_data)
m_press_predicted = m_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

In [46]:
df_regressed_3 = data.select(q_features)
df_regressed_3 = df_regressed_3.join(m_press_predicted, m_press_predicted.b_id == df_regressed_3.breath_ids).select(*df_regressed_3.columns, col('prediction').alias('m_pressure'))
df_regressed_3.show(5)

+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|R_5|R_20|C_10|C_20|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        q_pressure|        m_pressure|
+----------+---+----+----+----+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148|  0|   0|   1|   0| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 4.787830588906654|1.2455432212222992|
|       463|  0|   0|   1|   0| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274|14.032851614737082|2.9225295045356945|
|       471|  0|   1|   0|   1|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|  7.08478856513968|2.7090587869807883|
|       496|  0|   1|   0|   1| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 18.636305